# Molecular Oncology Almanac workshop
This notebook is being used as a demonstration for the [Cancer Genomics Consortium 2023](https://cancergenomics.org/meetings/cgc_annual_meeting_2023.php) Bioinformatics Workshop on Exploring the Clinical Interpretation Resource Landscape. It can be found on GitHub at [github.com/vanallenlab/2023-cgc-moalmanac](https://github.com/vanallenlab/2023-cgc-moalmanac) and you can learn more at [moalmanac.org](moalmanac.org).

**For this notebook, please move it to your moalmanac/moalmanac execution directory to import modules!**
https://github.com/vanallenlab/moalmanac

<img src="img/moalmanac-browser.png" alt="Molecular Oncology Almanac browser" width="750"/>

<a id='toc'></a>
## Table of contents
- <a href="#algorithm">Example algorithm execution</a>
    - <a href="#configure">Configuration</a>
    - <a href="#inputs">Specify inputs</a>
    - <a href="#run">Run algorithm</a>
    - <a href="#outputs">Review outputs</a>
- <a href="#conclusion">Conclusion</a>

<a id='algorithm'></a>
## Algorithm execution
The codebase for the Molecular Oncology Almanac algorithm can be found on [GitHub](https://github.com/vanallenlab/moalmanac) or [Docker Hub](https://hub.docker.com/r/vanallenlab/moalmanac/). 

<img src="img/moalmanac-algorithm-schematic.png" alt="Molecular Oncology Almanac browser" width="750"/>

# Please move this notebook to your moalmanac/moalmanac execution directory!
Please download and set up MOAlmanac (https://github.com/vanallenlab/moalmanac) for use on your local machine to run this notebook. **This notebook should be executed from your moalmanac/moalmanac directory**.

In [3]:
import sys
python_path = '/'.join(sys.executable.split('/')[:-1])
pip_path = f"{python_path}/pip"


virtual_env_name = "2023-cgc-workshop-moalmanac"
moalmanac_directory = '/Users/brendan/Github/moalmanac/moalmanac'

! $pip_path install -q -r $moalmanac_directory/requirements.txt

! cd /Users/brendan/Github/moalmanac/moalmanac

import moalmanac

<a id='configure'></a>
### Configure implementation
MOAlmanac contains two configuration files to customize of the algorithm's use. 
- `config.ini` allows users to change settings of the algorithm and toggle features
- `colnames.ini` allows users to change strings expected to be found within input data, internally to the algorithm, and written as outputs

In [12]:
from config import CONFIG
from config import COLNAMES

In [13]:
for key in CONFIG['function_toggle']:
    print(key)

calculate_model_similarity
calculate_mutational_signatures
calculate_preclinical_efficacy
generate_actionability_report
include_model_similarity_in_actionability_report
include_preclinical_efficacy_in_actionability_report
plot_mutational_signatures
plot_preclinical_efficacy


In [14]:
COLNAMES.keys()

dict_keys(['input_data', 'tcga_maf_input', 'gdc_maf_input', 'indel_input', 'germline_input', 'validation_input', 'seg_input', 'called_cn_input', 'fusion_input', 'simple_input', 'maps', 'patient', 'aneuploidy', 'features', 'burden', 'microsatellite', 'signatures', 'oncotree', 'validation_sequencing', 'overlap_somatic_germline', 'datasources', 'integrative', 'bin_names', 'report', 'outputs', 'preclinical', 'matchmaking'])

In [15]:
COLNAMES['germline_input']

{'gene': 'hugo_symbol',
 'chr': 'chromosome',
 'start': 'start_position',
 'end': 'end_position',
 'ref': 'reference_allele',
 'allele1': 'tumor_seq_allele1',
 'allele2': 'tumor_seq_allele2',
 'alt_type': 'variant_classification',
 'alt': 'protein_change',
 'tumor': 'tumor_sample_barcode',
 'normal': 'matched_norm_sample_barcode',
 'ref_count': 't_ref_count',
 'alt_count': 't_alt_count'}

<a id='inputs'></a>
### Specify inputs
The Molecular Oncology Almanac will annotate and evaluate any combination of inputs provided.Complete documentation of inputs accepted is [available within the algorithm's documentation](https://github.com/vanallenlab/moalmanac/blob/main/docs/description-of-inputs.md). 

In short for sample metadata,
- a profile id with `--patient_id` is the only required argument.
- a tumor type can be provided with `--tumor_type`, which will be mapped to [OncoTree](https://oncotree.mskcc.org/#/home) codes.  
- additional metadata can be passed with `--stage` and `--description`, both free text fields.

and for genomic alterations,
- somatic variants and germline variants can be provided with `--snv_handle`, `--indel_handle`, and `--germline_handle` as MAF files following either NCI or TCGA specification.
- tumor mutational burden will be calculated if the bases covered to call somatic variants is provided as an integer with the argument `--bases_covered_handle
- called gene-level copy number alterations can be provided with `--called_cn_handle` and fusions can be provided with `--fusion_handle`. 
- purity and ploidy can be provided as float values with `--purity` and `--ploidy` as well. 
- somatic variants called from validation sequencing can be provided with `--validation_handle`

Alternatively, a simplified input file that contains minimal variant details may be provided to the algorithm. Such as,

| feature_type | feature | alteration_type | alteration |
|---|---|---|---|
| Somatic Variant | BRAF | Missense | p.V600E |
| Copy Number | CDK4 | Amplification |  |
| Rearrangement | COL1A1 | Fusion | COL1A1--CITED4 |
| Germline Variant | BRCA2 | Frameshift | p.S1982fs |


Additional settings can be set by modifying the config.ini file and column names of provided input files may be modified by editing the colnames.ini file.

Required arguments:
```
    --patient_id            <string>    patient identifier
```

Optional arguments:
```
    --tumor_type            <string>    tumor ontology, default=Unknown
    --stage                 <string>    tumor stage, default=Unknown
    --snv_handle            <string>    handle for MAF file of somatic single nucleotide variants
    --indel_handle          <string>    handle for MAF file of somatic insertions and deletions
    --bases_covered_handle  <string>    handle for text file which contains the number of calcable somatic bases
    --called_cn_handle      <string>    handle for text file which contained genes and copy number calls, will be used over `--cnv_handle`
    --cnv_handle            <string>    handle for annotated seg file for somatic copy number
    --fusion_handle         <string>    handle for STAR fusion output, .final.abridged
    --germline_handle       <string>    handle for MAF file of germline single nucleotide variants and insertions and deletions
    --validation_handle     <string>    handle for MAF file of somatic single nucleotide variant called from validation sequencing
    --ms_status             <string>    microsatellite status as deemed by MSI sensor, MSI or MSS, default=Unknown
    --purity                <float>     tumor purity
    --ploidy                <float>     tumor ploidy
    --wgd                   <boolean>   specify the occurence of whole genome duplication
    --description           <string>    description of patient
    --output-directory      <string>    specify location of produced outputs
```


Here, we will utilize example data found within in this repository for use with moalmanac. 

In [16]:
metadata_dictionary = {
    'patient_id': 'example',
    'reported_tumor_type': 'SKCM',
    'stage': 'Metastatic',
    'description': 'Example data for CGC workshop!',
    'purity': 0.85,
    'ploidy': 4.02,
    'WGD': True,
    'microsatellite_status': 'msih'
}

root = "/Users/brendan/Github/scratch/2023-cgc-moalmanac/"
input_dictionary = {
    'snv_handle': '/Users/brendan/Github/scratch/2023-cgc-moalmanac/example-data/example_patient.capture.somatic.snvs.maf',
    'indel_handle': '/Users/brendan/Github/scratch/2023-cgc-moalmanac/example-data/example_patient.capture.somatic.indels.maf',
    'bases_covered_handle': '/Users/brendan/Github/scratch/2023-cgc-moalmanac/example-data/example_patient.capture.somatic.coverage.txt',
    'called_cn_handle': '/Users/brendan/Github/scratch/2023-cgc-moalmanac/example-data/example_patient.capture.somatic.called.cna.txt',
    'cnv_handle': '',
    'fusion_handle': '/Users/brendan/Github/scratch/2023-cgc-moalmanac/example-data/example_patient.rna.star.fusions.txt',
    'germline_handle': '/Users/brendan/Github/scratch/2023-cgc-moalmanac/example-data/example_patient.capture.germline.maf',
    'validation_handle': '/Users/brendan/Github/scratch/2023-cgc-moalmanac/example-data/example_patient.rna.somatic.snvs.maf',
    'disable_matchmaking': True
}

<a id='run'></a>
### Run algorithm
The Molecular Oncology Almanac algorithm can be executed by running the Python code from Bash, from within a Python environment by instantiating the main function, or by utilizing or modifying the [example run script](https://github.com/vanallenlab/moalmanac/blob/main/moalmanac/run_example.py). Please execute [moalmanac/simplified_input.py](https://github.com/vanallenlab/moalmanac/tree/main/moalmanac#simplified-input) if performing interpretation on a simpified input. Otherwise, an example execution may appear as,

```
python moalmanac.py \
    --patient_id "example" \
    --tumor_type "SKCM" \
    --stage "Metastatic" \
    --description "Example profile for interpretation with the Molecular Oncology Almanac" \
    --snv_handle "example-data/example_patient.capture.somatic.snvs.maf" \
    --indel_handle "example-data/example_patient.capture.somatic.indels.maf" \
    --bases_covered_handle "example-data/example_patient.capture.somatic.coverage.txt" \
    --called_cn_handle "example-data/example_patient.capture.somatic.called.cna.txt" \
    --fusion_handle "example-data/example_patient.capture.somatic.seg.annotated" \
    --germline_handle "example-data/example_patient.rna.star.fusions.txt" \
    --validation_handle "example-data/example_patient.rna.somatic.snvs.maf" \
    --purity 0.85 \
    --ploidy 4.02 \
    --ms_status "msih" \
    --wgd
```

Interpretation on clinically or biologically relevant genomic alterations cause the computational run time to scale linearly. Using the example data in this repository, performing just the somatic and germline interpretation takes ~90 seconds to complete (19 clinically or biologically relevant variants, along with the rest of the provided variants). This time will increase if you enable: report generation, preclinical efficacy, similarity comparisons, and mutational signature calculations. 

In [17]:
output_directory = "workshop-outputs/"
!mkdir -p $output_directory

import time
start_time = time.time()

moalmanac.main(metadata_dictionary, input_dictionary, output_directory)

end_time = time.time()
time_statement = "Molecular Oncology Almanac runtime: %s seconds" % round((end_time - start_time), 4)
print(time_statement)

Molecular Oncology Almanac runtime: 92.05 seconds


<a id='outputs'></a>
### Review outputs
The Molecular Oncology Almanac has the ability to produce several outputs, dependant on which inputs are provided. In all cases, a [fully-contained interactive report](example-outputs/example.report.html) will be generated. Complete documentation of outputs produced is [available within the algorithm's documentation](https://github.com/vanallenlab/moalmanac/blob/main/docs/description-of-outputs.md).

In [18]:
import pandas as pd

Genomic alterations that are associated with clinical or biological significance are highlighted in the `.actionable.txt` output. These are the same findings that will be reported within the [interactive report](example-outputs/example.report.html).

This output contains one row for each genomic alteration highlighted along with associated relevance for therapeutic sensitivity, resistance, or disease prognosis.

In [19]:
file = 'example-outputs/example.actionable.txt'
df = pd.read_csv(file, sep='\t')
df.head()

,score_bin,sensitive_predictive_implication,resistance_predictive_implication,prognostic_predictive_implication,feature_type,feature,alteration_type,alteration,tumor_f,total_coverage,...,prognostic_url,number_germline_mutations_in_gene,validation_total_coverage,validation_tumor_f,validation_detection_power,feature_display,preclinical_efficacy_observed,patient_id,tumor_sample_barcode,normal_sample_barcode
0,Putatively Actionable,FDA-Approved,Guideline,Guideline,Somatic Variant,BRAF,Missense,p.V600E,0.6316,152.0,...,https://www.nccn.org/professionals/physician_g...,1.0,4.0,0.0,0.161,BRAF p.V600E (Missense),1.0,example,example_tumor_profile,example_normal_profile
1,Investigate Actionability,FDA-Approved,Guideline,Guideline,Microsatellite Stability,MSI-High,NaN,NaN,NaN,NaN,...,https://www.nccn.org/professionals/physician_g...,NaN,NaN,NaN,NaN,MSI-High,NaN,example,NaN,NaN
2,Investigate Actionability,Guideline,NaN,NaN,Rearrangement,COL1A1,Fusion,COL1A1--CITED4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,COL1A1--CITED4 Fusion,0.0,example,NaN,NaN
3,Investigate Actionability,NaN,NaN,Inferential,Aneuploidy,Whole genome doubling,NaN,NaN,NaN,NaN,...,https://doi.org/10.1038/s41588-018-0165-1,NaN,NaN,NaN,NaN,NaN,NaN,example,NaN,NaN
4,Investigate Actionability,FDA-Approved,NaN,NaN,Germline Variant,BRCA2,Frameshift,p.S1982fs,0.5000,100.0,...,NaN,NaN,NaN,NaN,NaN,BRCA2 p.S1982fs (Frameshift),NaN,example,example_tumor_profile,example_normal_profile


Somatic nucleotide variants (`--snv_handle` and (`--indel_handle`), copy number alterations (`--called_cn_handle`), and fusions (`--`) are summarized in the the `.somatic.scored.txt` output. This output reports a broad clinical interpretation and biological interpretation by presence or absence in several datasources. 

In [20]:
file = 'example-outputs/example.somatic.scored.txt'
df = pd.read_csv(file, sep='\t')
df.head()

,score_bin,sensitive_score_bin,resistance_score_bin,prognostic_score_bin,feature_type,feature,alteration_type,alteration,chromosome,start_position,...,patient_id,tumor_sample_barcode,normal_sample_barcode,almanac_bin,cancerhotspots_bin,cancerhotspots3D_bin,cgc_bin,gsea_pathways_bin,gsea_modules_bin,cosmic_bin
0,Putatively Actionable,Putatively Actionable,Putatively Actionable,Putatively Actionable,Somatic Variant,BRAF,Missense,p.V600E,7,140453136.0,...,example,example_tumor_profile,example_normal_profile,4,2,2,1,1,1,2
1,Putatively Actionable,Putatively Actionable,NaN,NaN,Copy Number,CDK4,Amplification,NaN,NaN,NaN,...,example,NaN,NaN,4,1,2,1,1,1,1
2,Putatively Actionable,NaN,NaN,Putatively Actionable,Copy Number,TP53,Deletion,NaN,NaN,NaN,...,example,NaN,NaN,4,1,1,1,1,1,1
3,Putatively Actionable,NaN,Putatively Actionable,NaN,Copy Number,BRAF,Amplification,NaN,NaN,NaN,...,example,NaN,NaN,4,1,1,1,1,1,1
4,Putatively Actionable,Putatively Actionable,NaN,NaN,Copy Number,CDKN2A,Deletion,NaN,NaN,NaN,...,example,NaN,NaN,4,1,0,1,1,1,1


If germline variants are provided using the `--germline_handle`, outputs will be produced to highlight variants in genes: 
- listed by the American College of Medical Genetics, `.germline.acmg.txt`
- associated with somatic cancers, `.germline.cancer_related.txt`
- and hereditary cancer risk, `.germline.hereditary_cancer.txt` 

Common variants and variants labeled as benign or likely benign by ClinVar are _not_ included in these outputs.

In [21]:
file = 'example-outputs/example.germline.acmg.txt'
df = pd.read_csv(file, sep='\t')
df.head()

,feature_type,feature,alteration_type,alteration,chromosome,start_position,end_position,reference_allele,observed_allele1,observed_allele2,...,exac_afr_an,exac_amr_an,exac_eas_an,exac_fin_an,exac_nfe_an,exac_sas_an,exac_oth_an,patient_id,tumor_sample_barcode,normal_sample_barcode
0,Germline Variant,MSH6,Missense,p.G39E,2,48010488,48010488,G,G,A,...,6388.0,9014.0,6792.0,5078.0,48404.0,14790.0,664.0,example,example_tumor_profile,example_normal_profile
1,Germline Variant,TP53,Missense,p.P72R,17,7579472,7579472,G,G,C,...,10030.0,11562.0,8620.0,6610.0,66692.0,16506.0,904.0,example,example_tumor_profile,example_normal_profile
2,Germline Variant,BRCA2,Missense,p.N372H,13,32906729,32906729,A,A,C,...,9854.0,11568.0,8646.0,6614.0,66692.0,16490.0,904.0,example,example_tumor_profile,example_normal_profile
3,Germline Variant,BRCA2,Frameshift,p.S1982fs,13,32914438,32914438,T,T,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,example,example_tumor_profile,example_normal_profile


Observed somatic nucleotide variants, germline variants, copy number alterations, and fusions will be combined by gene in the integrated summary output, `.integrated.summary.txt`, to facilitate integrative analyses. 

In [22]:
file = 'example-outputs/example.integrated.summary.txt'
df = pd.read_csv(file, sep='\t')
df.head()

,feature,somatic_mutations,copy_number_alterations,fusions,germline_mutations
0,ALK,NaN,NaN,NaN,Missense p.I1461V
1,BCR,NaN,NaN,NaN,Missense p.N796S
2,BLM,NaN,Amplification,NaN,NaN
3,BRAF,Missense p.V600E,NaN,NaN,Nonsense p.R509*
4,BRCA2,NaN,NaN,NaN,"Missense p.N372H, Frameshift p.S1982fs"


Variants associated with microsatellite stability will also be extracted and summarized, `msi_variants.txt`. As of the current algorithm release, the following genes are aggregated,
- _DOCK3_
- _ESRP1_
- _MLH1_
- _MSH2_
- _MSH6_
- _PMS2_
- _POLE_
- _POLE2_
- _PRDM2_

In [23]:
file = 'example-outputs/example.msi_variants.txt'
df = pd.read_csv(file, sep='\t')
df.head()

,score_bin,sensitive_score_bin,resistance_score_bin,prognostic_score_bin,feature_type,feature,alteration_type,alteration,chromosome,start_position,...,patient_id,tumor_sample_barcode,normal_sample_barcode,almanac_bin,cancerhotspots_bin,cancerhotspots3D_bin,cgc_bin,gsea_pathways_bin,gsea_modules_bin,cosmic_bin
0,Biologically Relevant,NaN,NaN,NaN,Copy Number,JAK1,Deletion,NaN,1,65047989,...,example,example_tumor_profile,example_normal_profile,1,1,0.0,1,1.0,1.0,1.0
1,Cancer Gene Census,NaN,NaN,NaN,Germline Variant,PRDM2,Deletion,p.E282del,1,14105122,...,example,example_tumor_profile,example_normal_profile,0,0,NaN,1,NaN,NaN,NaN


Tumor mutational burden will be calculated if somatic variants and the bases considered are provided, `.mutational_burden.txt`. Here, the nonsynyonmous mutational burden or coding mutational burden is calculated. Additionally, the Molecular Oncology Almanac will compare the observed mutational burden to TCGA.

Tumor mutational burden will be evaluated as high if two conditions are met,
- Mutations per megabase (Mb) > 10 
- At least a mutational burden of 80th percentile of TCGA tumor type, if matched to a TCGA tumor type, else TCGA generally.

In [24]:
file = 'example-outputs/example.mutational_burden.txt'
df = pd.read_csv(file, sep='\t')
df.loc[0, :]

patient_id                                          example
reported_tumor_type                                    SKCM
oncotree_term                            Cutaneous Melanoma
oncotree_code                                          SKCM
bases_covered                                    29908096.0
n_nonsyn_mutations                                      204
coding_mutational_burden_per_megabase                 6.821
percentile_tcga                                       83.75
percentile_tcga_tissuetype                            25.62
high_burden?                                          False
Name: 0, dtype: object

Trinucleotide mutational signatures will be calculated using a subprocess if somatic variants are provided and the feature is enabled. The following relevant outputs are produced,
- `.sigs.context.txt`, variant counts for each 96 trinucleotide bins
- `.sigs.cosmic.txt`, weights for each of the COSMIC (v2) mutational signatures
- `.sigs.tricontext.counts.png`, visualization of trinucleotide context counts of observed somatic variants
- `.sigs.tricontext.normalized.png`, visualization of normalized counts of somatic variants across trinucleotide contexts

In [25]:
file = 'example-outputs/example.sigs.context.txt'
df = pd.read_csv(file, sep='\t')
df.head()

,A[C>A]A,A[C>A]C,A[C>A]G,A[C>A]T,C[C>A]A,C[C>A]C,C[C>A]G,C[C>A]T,G[C>A]A,G[C>A]C,...,C[T>G]G,C[T>G]T,G[T>G]A,G[T>G]C,G[T>G]G,G[T>G]T,T[T>G]A,T[T>G]C,T[T>G]G,T[T>G]T
0,0,2,1,0,3,1,1,1,0,0,...,0,3,0,0,0,0,0,1,0,0


<img src="example-outputs/example.sigs.tricontext.counts.png" alt="Variant counts by trinucleotide context" width="750"/>


<a href="#toc">Return to Table of Contents</a>

<a id='conclusion'></a>
## In conclusion

**Thank you very much for inviting us to give this workshop today, please reach out with any additional questions or comments!**

- Email: moalmanac@ds.dfci.harvard.edu
- Twitter: [@moalmanac](https://twitter.com/moalmanac), [@vanallenlab](https://twitter.com/vanallenlab)

Slides and code from this workshop are available on GitHub at [https://github.com/vanallenlab/2023-cgc-moalmanac](https://github.com/vanallenlab/2023-cgc-moalmanac).

<img src="img/conclusion-slide.png" alt="Thank you!" width="750"/>

Molecular Oncology Almanac (MOAlmanac) is a clinical interpretation algorithm paired with an underlying knowledge base for precision oncology. The primary objective of MOAlmanac is to identify and associate molecular alterations with therapeutic sensitivity and resistance as well as disease prognosis. This is done for “first-order” genomic alterations -- individual events such as somatic variants, copy number alterations, fusions, and germline -- as well as “second-order” events -- those that are not associated with one single mutation, and may be descriptive of global processes in the tumor such as tumor mutational burden, microsatellite instability, mutational signatures, and whole-genome doubling. In addition to clinical insights, MOAlmanac will annotate and evaluate first-order events based on their presence in numerous other well established datasources as well as highlight connections between them. This method currently geared towards hg19/b37 reference files and whole-exome or targeted sequencing data.

There are several other resources within [the Molecular Oncology Almanac ecosystem](https://github.com/topics/molecular-oncology-almanac): 
- Molecular Oncology Almanac, the clinical interpretation algorithm for precision cancer medicine [[GitHub](https://github.com/vanallenlab/moalmanac)].
- [Molecular Oncology Almanac Browser](https://moalmanac.org), a website to browse our underlying knowledge base [[GitHub](https://github.com/vanallenlab/moalmanac-browser)].
- [Molecular Oncology Almanac Connector](https://chrome.google.com/webstore/detail/molecular-oncology-almana/jliaipolchffpaccagodphgjpfdpcbcm?hl=en), a Google Chrome extension to quickly suggest literature for cataloging [[GitHub](https://github.com/vanallenlab/moalmanac-extension)].
- Molecular Oncology Almanac Database, the content and release notes of our underlying knowledge base [[GitHub](https://github.com/vanallenlab/moalmanac-db)].
- [Molecular Oncology Almanac Portal](https://portal.moalmanac.org), a website to launch this method on the Broad Institute's Google Cloud platform called Terra [[GitHub](https://github.com/vanallenlab/moalmanac-portal)].

This method is also available on [Docker](https://hub.docker.com/repository/docker/vanallenlab/moalmanac) and [Terra](https://portal.firecloud.org/#methods/vanallenlab/moalmanac/). We have also released [code on GitHub to help facilitate analyses of multiple samples that have been interpreted using the Molecular Oncology Almanac](https://github.com/vanallenlab/moalmanac-cohort). 

If you use this method, please cite our publication:
> [Reardon, B., Moore, N.D., Moore, N.S., *et al*. Integrating molecular profiles into clinical frameworks through the Molecular Oncology Almanac to prospectively guide precision oncology. *Nat Cancer* (2021). https://doi.org/10.1038/s43018-021-00243-3](https://www.nature.com/articles/s43018-021-00243-3)
